# Introduction

In [1]:
import pySSV
import logging
pySSV.ssv_logging.set_severity(logging.DEBUG)

In [8]:
w = pySSV.SSVRenderWidget()
w

SSVRenderWidget()

In [9]:
import pySSV as ssv

canvas = ssv.canvas()
canvas.dbg_log_context()

[pySSV] [INFO] [ssv_logging] [log] Got OpenGL context:
	GL_VENDOR=NVIDIA Corporation
	GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
	GL_VERSION=3.3.0 NVIDIA 537.42


In [12]:
# canvas.run()